# Topic Modeling with gensim

In [ ]:
# gensim
import gensim
from gensim import corpora, models, similarities, matutils
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# sklearn
from sklearn import datasets
from sklearn.feature_extraction.text import CountVectorizer

# logging for gensim (set to INFO)
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

import pickle
import numpy as np
import pandas as pd

## Document Preprocessing
We'll need to generate a term-document matrix of word (token) counts for use in LDA.

We'll use `sklearn`'s `CountVectorizer` to generate our term-document matrix of counts. We'll make use of a few parameters to accomplish the following preprocessing of the text documents all within the `CountVectorizer`:
* `analyzer=word`: Tokenize by word
* `ngram_range=(1,2)`: Keep all 1 and 2-word grams
* `stop_words=english`: Remove all English stop words
* `token_pattern=\\b[a-z][a-z]+\\b`: Match all tokens with 2 or more (strictly) alphabet characters

In [ ]:
#Open Corpus of News Article Text
# with open('./data/news_df.pickle', 'rb') as file:
with open('./data/news_data_frame_reduced_preprocessed.pickle', 'rb') as file:
# with open('./data/news_fake_real_df_reduced_token.pickle', 'rb') as file:
    news_data_frame = pickle.load(file)

In [ ]:
news_data_frame["reshaped_tokenized_text"] = news_data_frame["tokenized_text"].apply(lambda x: ' '.join(x))

In [ ]:
#Remove Overfit Words
import re
said_remove = re.compile('(\s*)said(\s*)')
facebook_remove = re.compile('(\s*)facebook(\s*)')
obama_remove = re.compile('(\s*)obama(\s*)')
trump_remove = re.compile('(\s*)trump(\s*)')
hillary_remove = re.compile('(\s*)hillary(\s*)')
clinton_remove = re.compile('(\s*)clinton(\s*)')

news_data_frame["reshaped_tokenized_text"] = news_data_frame["reshaped_tokenized_text"].apply(lambda x: said_remove.sub('', x))
news_data_frame["reshaped_tokenized_text"] = news_data_frame["reshaped_tokenized_text"].apply(lambda x: facebook_remove.sub('', x))
news_data_frame["reshaped_tokenized_text"] = news_data_frame["reshaped_tokenized_text"].apply(lambda x: obama_remove.sub('', x))
news_data_frame["reshaped_tokenized_text"] = news_data_frame["reshaped_tokenized_text"].apply(lambda x: trump_remove.sub('', x))
news_data_frame["reshaped_tokenized_text"] = news_data_frame["reshaped_tokenized_text"].apply(lambda x: hillary_remove.sub('', x))
news_data_frame["reshaped_tokenized_text"] = news_data_frame["reshaped_tokenized_text"].apply(lambda x: clinton_remove.sub('', x))


In [ ]:
real_news_data = news_data_frame.loc[news_data_frame['Not_Real_or_Real'] == 0, 'reshaped_tokenized_text']
fake_news_data = news_data_frame.loc[news_data_frame['Not_Real_or_Real'] == 1, 'reshaped_tokenized_text']
all_news_data =  news_data_frame['reshaped_tokenized_text']

In [ ]:
# Create a CountVectorizer for parsing/counting words
count_vectorizer = CountVectorizer(ngram_range=(1, 2),  
                                   stop_words='english', token_pattern="\\b[a-z][a-z]+\\b")

count_vectorizer_real = count_vectorizer.fit(real_news_data)
count_vectorizer_fake = count_vectorizer.fit(real_news_data)
count_vectorizer_all = count_vectorizer.fit(real_news_data)

In [ ]:
# Create the term-document matrix
# Transpose it so the terms are the rows
doc_word_real_news = count_vectorizer_real.transform(real_news_data).transpose()
doc_word_fake_news = count_vectorizer_fake.transform(fake_news_data).transpose()
doc_word_all_news = count_vectorizer_all.transform(all_news_data).transpose()

In [ ]:
print(doc_word_real_news.shape)
print(doc_word_fake_news.shape)
print(doc_word_all_news.shape)

##### Convert to gensim
We need to convert our sparse `scipy` matrix to a `gensim`-friendly object called a Corpus:

In [ ]:
# Convert sparse matrix of counts to a gensim corpus
corpus_real_news = matutils.Sparse2Corpus(doc_word_real_news)
corpus_fake_news = matutils.Sparse2Corpus(doc_word_fake_news)
corpus_all_news = matutils.Sparse2Corpus(doc_word_all_news)

##### Map matrix rows to words (tokens)
We need to save a mapping (dict) of row id to word (token) for later use by gensim:

In [ ]:
id2word_real = dict((v, k) for k, v in count_vectorizer_real.vocabulary_.items())
id2word_fake = dict((v, k) for k, v in count_vectorizer_fake.vocabulary_.items())
id2word_all = dict((v, k) for k, v in count_vectorizer_all.vocabulary_.items())

## LDA
At this point we can simply plow ahead in creating an LDA model.  It requires our corpus of word counts, mapping of row ids to words, and the number of topics (3).

### Real News

In [ ]:
# Create lda model (equivalent to "fit" in sklearn)
lda_real = models.LdaModel(corpus=corpus_real_news, num_topics=3, id2word=id2word_real, passes=20)

### Fake News

In [ ]:
# Create lda model (equivalent to "fit" in sklearn)
lda_fake = models.LdaModel(corpus=corpus_fake_news, num_topics=3, id2word=id2word_fake, passes=5)

### All News

In [ ]:
# Create lda model (equivalent to "fit" in sklearn)
lda_all = models.LdaModel(corpus=corpus_all_news, num_topics=3, id2word=id2word_all, passes=5)

Let's take a look at what happened.  Here are the 5 most important words for each of the 3 topics we found:

In [ ]:
lda.print_topics()

#### Topic Space
If we want to map our documents to the topic space we need to actually use the LdaModel transformer that we created above, like so:

In [ ]:
# Transform the docs from the word space to the topic space (like "transform" in sklearn)
lda_corpus = lda[corpus]
lda_corpus

In [ ]:
# Store the documents' topic vectors in a list so we can take a peak
lda_docs = [doc for doc in lda_corpus]

Now we can take a look at the document vectors in the topic space, which are measures of the component of each document along each topic.  Thus, at most a document vector can have num_topics=3 nonzero components in the topic space, and most have far fewer.

In [ ]:
# Check out the document vectors in the topic space for the first 5 documents
lda_docs[0:5]

In [ ]:
ng_train.data[0]

## LDA Visualizaiton Using Tri and Bi Grams

In [ ]:
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [ ]:
fileObject = open('./data/grams/bigram_text','rb')  
bi_lem_comb2 = pickle.load(fileObject)

In [ ]:
fileObject = open('./data/grams/trigram_text','rb')  
tri_lem_comb2 = pickle.load(fileObject)

In [ ]:
bi_gram_series = pd.Series(bi_lem_comb2) 

In [ ]:
#Tokenize Bigrams
from nltk.tokenize import word_tokenize
bi_gram_series = bi_gram_series.apply(word_tokenize)

In [ ]:
tri_gram_series = pd.Series(tri_lem_comb2) 

In [ ]:
#Tokenize Bigrams
tri_gram_series = tri_gram_series.apply(word_tokenize)

In [ ]:
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()

#Lemmatizing Text
tri_gram_lemmatized = tri_gram_series.apply(lambda x: [lemmatizer.lemmatize(y) for y in x])

#Lemmatizing Text
bi_gram_lemmatized = bi_gram_series.apply(lambda x: [lemmatizer.lemmatize(y) for y in x])

In [ ]:
id2word_tri = corpora.Dictionary(tri_gram_lemmatized)
id2word_bi = corpora.Dictionary(bi_gram_lemmatized)

In [ ]:
# Create Corpus
text_tri = tri_gram_lemmatized
text_bi  = bi_gram_lemmatized

In [ ]:
# Term Document Frequency
corpus_tri = [id2word_tri.doc2bow(text) for text in text_tri]
corpus_bi = [id2word_bi.doc2bow(text) for text in text_bi]

### Note
Gensim creates a unique id for each word in the document. The produced corpus shown above is a mapping of (word_id, word_frequency).

For example, (0, 1) above implies, word id 0 occurs once in the first document. Likewise, word id 1 occurs twice and so on.

This is used as the input by the LDA model.

If you want to see what word a given id corresponds to, pass the id as a key to the dictionary.

In [ ]:
# Human readable format of corpus (term-frequency)
[[(id2word_tri[id], freq) for id, freq in cp] for cp in corpus_tri[:1]]

# Building the Topic Model
In addition to the corpus and dictionary, will need to provide the number of topics as well.

Apart from that, alpha and eta are hyperparameters that affect sparsity of the topics. According to the Gensim docs, both defaults to 1.0/num_topics prior.

chunksize is the number of documents to be used in each training chunk. update_every determines how often the model parameters should be updated and passes is the total number of training passes.

In [ ]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus_tri,
                                           id2word=id2word_tri,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

## View the topics in LDA model
The above LDA model is built with 20 different topics where each topic is a combination of keywords and each keyword contributes a certain weightage to the topic.

You can see the keywords for each topic and the weightage(importance) of each keyword using lda_model.print_topics() as shown next.

In [ ]:
# Print the Keyword in the 10 topics
print(lda_model.print_topics())
doc_lda = lda_model[corpus_tri]

## Compute Model Perplexity and Coherence Score
Model perplexity and topic coherence provide a convenient measure to judge how good a given topic model is. In my experience, topic coherence score, in particular, has been more helpful.

In [ ]:
### Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus_tri))  # a measure of how good the model is. lower the better.

### Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=tri_gram_lemmatized, dictionary=id2word_tri, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

# Visualize the topics-keywords

In [ ]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus_tri, id2word_tri)
vis

# pyLDAvis.sklearn

In [1]:
from __future__ import print_function
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

import pickle

In [2]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [3]:
# gensim
import gensim
from gensim import corpora, models, similarities, matutils
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

from gensim import corpora, models, similarities, matutils
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from gensim.models import Phrases
from gensim.models.phrases import Phraser

# sklearn
from sklearn import datasets
from sklearn.feature_extraction.text import CountVectorizer

# logging for gensim (set to INFO)
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

import pickle
import numpy as np
import pandas as pd

C:\Users\moham\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Users\moham\Anaconda3\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [44]:
#Open Corpus of News Article Text
with open('./data/news_data_frame_reduced_preprocessed.pickle', 'rb') as file:
    news_data_frame = pickle.load(file)

In [45]:
news_data_frame["reshaped_tokenized_text"] = news_data_frame["tokenized_text"].apply(lambda x: ' '.join(x))

In [46]:
#Remove Overfit Words
import re
said_remove = re.compile(r'(\s*)said(\s*)')
facebook_remove = re.compile(r'(\s*)facebook(\s*)')
obama_remove = re.compile(r'(\s*)obama(\s*)')
trump_remove = re.compile(r'(\s*)trump(\s*)')
hillary_remove = re.compile(r'(\s*)hillary(\s*)')
clinton_remove = re.compile(r'(\s*)clinton(\s*)')

news_data_frame["reshaped_tokenized_text"] = news_data_frame["reshaped_tokenized_text"].apply(lambda x: said_remove.sub(' ', x))
news_data_frame["reshaped_tokenized_text"] = news_data_frame["reshaped_tokenized_text"].apply(lambda x: facebook_remove.sub(' ', x))
news_data_frame["reshaped_tokenized_text"] = news_data_frame["reshaped_tokenized_text"].apply(lambda x: obama_remove.sub(' ', x))
news_data_frame["reshaped_tokenized_text"] = news_data_frame["reshaped_tokenized_text"].apply(lambda x: trump_remove.sub(' ', x))
news_data_frame["reshaped_tokenized_text"] = news_data_frame["reshaped_tokenized_text"].apply(lambda x: hillary_remove.sub(' ', x))
news_data_frame["reshaped_tokenized_text"] = news_data_frame["reshaped_tokenized_text"].apply(lambda x: clinton_remove.sub(' ', x))


In [47]:
#Tokenize Text
from nltk.tokenize import word_tokenize
news_data_frame["reshaped_tokenized_text"] = news_data_frame["reshaped_tokenized_text"].apply(word_tokenize)

In [48]:
real_news_data = news_data_frame.loc[news_data_frame['Not_Real_or_Real'] == 0, 'reshaped_tokenized_text']
fake_news_data = news_data_frame.loc[news_data_frame['Not_Real_or_Real'] == 1, 'reshaped_tokenized_text']
all_news_data =  news_data_frame['reshaped_tokenized_text']

### Prepare Bigram Text

In [49]:
bigram_real = Phrases(real_news_data, min_count=5, threshold = 80);
bigram_mod_real = Phraser(bigram_real);

2019-05-27 17:30:15,120 : INFO : collecting all words and their counts
2019-05-27 17:30:15,121 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2019-05-27 17:30:16,606 : INFO : collected 578030 word types from a corpus of 1061812 words (unigram + bigrams) and 2314 sentences
2019-05-27 17:30:16,608 : INFO : using 578030 counts as vocab in Phrases<0 vocab, min_count=5, threshold=80, max_vocab_size=40000000>
2019-05-27 17:30:16,624 : INFO : source_vocab length 578030
2019-05-27 17:30:21,660 : INFO : Phraser built with 6561 6561 phrasegrams


In [50]:
bigram_fake = Phrases(fake_news_data, min_count=5, threshold = 80);
bigram_mod_fake = Phraser(bigram_fake);

2019-05-27 17:30:21,665 : INFO : collecting all words and their counts
2019-05-27 17:30:21,666 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2019-05-27 17:30:22,681 : INFO : collected 518114 word types from a corpus of 707360 words (unigram + bigrams) and 2112 sentences
2019-05-27 17:30:22,682 : INFO : using 518114 counts as vocab in Phrases<0 vocab, min_count=5, threshold=80, max_vocab_size=40000000>
2019-05-27 17:30:22,693 : INFO : source_vocab length 518114
2019-05-27 17:30:27,102 : INFO : Phraser built with 2266 2266 phrasegrams


In [51]:
bigram_all = Phrases(all_news_data, min_count=5, threshold = 80);
bigram_mod_all = Phraser(bigram_all);

2019-05-27 17:30:27,109 : INFO : collecting all words and their counts
2019-05-27 17:30:27,109 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2019-05-27 17:30:29,866 : INFO : collected 1019403 word types from a corpus of 1769172 words (unigram + bigrams) and 4426 sentences
2019-05-27 17:30:29,867 : INFO : using 1019403 counts as vocab in Phrases<0 vocab, min_count=5, threshold=80, max_vocab_size=40000000>
2019-05-27 17:30:29,893 : INFO : source_vocab length 1019403
2019-05-27 17:30:38,759 : INFO : Phraser built with 8297 8297 phrasegrams


In [52]:
def make_bigrams(texts, type='real'):
    if type == 'real':
        return [bigram_mod_real[doc] for doc in texts]
    elif type == 'fake':
        return [bigram_mod_fake[doc] for doc in texts]
    else:
        return [bigram_mod_all[doc] for doc in texts]

In [53]:
words_bigrams_real = make_bigrams(real_news_data, 'real')
words_bigrams_fake = make_bigrams(real_news_data, 'fake')
words_bigrams_all = make_bigrams(real_news_data, 'all')

### Prepare Trigram Text

In [54]:
trigram_real = Phrases(real_news_data, threshold = 80);
trigram_mod_real = Phraser(trigram_real);

2019-05-27 17:30:45,865 : INFO : collecting all words and their counts
2019-05-27 17:30:45,866 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2019-05-27 17:30:47,444 : INFO : collected 578030 word types from a corpus of 1061812 words (unigram + bigrams) and 2314 sentences
2019-05-27 17:30:47,444 : INFO : using 578030 counts as vocab in Phrases<0 vocab, min_count=5, threshold=80, max_vocab_size=40000000>
2019-05-27 17:30:47,463 : INFO : source_vocab length 578030
2019-05-27 17:30:52,789 : INFO : Phraser built with 6561 6561 phrasegrams


In [55]:
trigram_fake = Phrases(fake_news_data, threshold = 80);
trigram_mod_fake = Phraser(trigram_fake);

2019-05-27 17:30:52,796 : INFO : collecting all words and their counts
2019-05-27 17:30:52,797 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2019-05-27 17:30:54,116 : INFO : collected 518114 word types from a corpus of 707360 words (unigram + bigrams) and 2112 sentences
2019-05-27 17:30:54,117 : INFO : using 518114 counts as vocab in Phrases<0 vocab, min_count=5, threshold=80, max_vocab_size=40000000>
2019-05-27 17:30:54,137 : INFO : source_vocab length 518114
2019-05-27 17:30:59,277 : INFO : Phraser built with 2266 2266 phrasegrams


In [56]:
trigram_all = Phrases(all_news_data, threshold = 80);
trigram_mod_all = Phraser(trigram_all);

2019-05-27 17:30:59,282 : INFO : collecting all words and their counts
2019-05-27 17:30:59,283 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2019-05-27 17:31:02,521 : INFO : collected 1019403 word types from a corpus of 1769172 words (unigram + bigrams) and 4426 sentences
2019-05-27 17:31:02,522 : INFO : using 1019403 counts as vocab in Phrases<0 vocab, min_count=5, threshold=80, max_vocab_size=40000000>
2019-05-27 17:31:02,550 : INFO : source_vocab length 1019403
2019-05-27 17:31:12,481 : INFO : Phraser built with 8297 8297 phrasegrams


In [57]:
def make_trigrams(texts, type='real'):
    if type == 'real':
        return [trigram_mod_real[doc] for doc in texts]
    elif type == 'fake':
        return [trigram_mod_fake[doc] for doc in texts]
    else:
        return [trigram_mod_all[doc] for doc in texts]

In [82]:
words_trigrams_real = pd.Series(make_trigrams(real_news_data, 'real'))
words_trigrams_fake = pd.Series(make_trigrams(real_news_data, 'fake'))
words_trigrams_all = pd.Series(make_trigrams(real_news_data, 'all'))

In [83]:
real_extracted = words_trigrams_real.apply(lambda x: ' '.join(x))
fake_extracted = words_trigrams_fake.apply(lambda x: ' '.join(x))
all_extracted = words_trigrams_all.apply(lambda x: ' '.join(x))

## Convert to document-term matrix
Next, the raw documents are converted into document-term matrix, possibly as raw counts or in TF-IDF form.

In [93]:
#Bigrams
tf_vectorizer_bi = CountVectorizer(strip_accents = 'unicode',
                                stop_words = 'english',
                                lowercase = True,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                max_df = 0.5, 
                                min_df = 10)

dtm_tf_bi = tf_vectorizer_bi.fit_transform(all_extracted)
print(dtm_tf_bi.shape)

tfidf_vectorizer_bi = TfidfVectorizer(**tf_vectorizer_bi.get_params())
dtm_tfidf_bi = tfidf_vectorizer_bi.fit_transform(all_extracted)
print(dtm_tfidf_bi.shape)

(2314, 8389)


C:\Users\moham\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1616: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. <class 'numpy.int64'> 'dtype' will be converted to np.float64.
  UserWarning)


(2314, 8389)


In [109]:
#Trigrams
tf_vectorizer_tri = CountVectorizer(strip_accents = 'unicode',
                                stop_words = 'english',
                                lowercase = True,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                max_df = 0.5, 
                                min_df = 10)

dtm_tf_tri = tf_vectorizer_tri.fit_transform(all_extracted)
print(dtm_tf_tri.shape)

tfidf_vectorizer_tri = TfidfVectorizer(**tf_vectorizer_tri.get_params())
dtm_tfidf_tri = tfidf_vectorizer_tri.fit_transform(all_extracted)
print(dtm_tfidf_tri.shape)

(2314, 8389)


C:\Users\moham\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1616: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. <class 'numpy.int64'> 'dtype' will be converted to np.float64.
  UserWarning)


(2314, 8389)


## Fit Latent Dirichlet Allocation models
Finally, the LDA models are fitted.

In [94]:
#Bigrams
# for TF DTM
lda_tf_bi = LatentDirichletAllocation(n_components=20, random_state=0)
lda_tf_bi.fit(dtm_tf_bi)

# for TFIDF DTM
lda_tfidf_bi = LatentDirichletAllocation(n_components=20, random_state=0)
lda_tfidf_bi.fit(dtm_tfidf_bi)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=20, n_jobs=None,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [110]:
#Trigrames
# for TF DTM
lda_tf_tri = LatentDirichletAllocation(n_components=20, random_state=0)
lda_tf_tri.fit(dtm_tf_tri)

# for TFIDF DTM
lda_tfidf_tri = LatentDirichletAllocation(n_components=20, random_state=0)
lda_tfidf_tri.fit(dtm_tfidf_tri)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=20, n_jobs=None,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

## Visualizing the models with pyLDAvis

In [89]:
bi_real = pyLDAvis.sklearn.prepare(lda_tf_bi, dtm_tf_bi, tf_vectorizer_bi)
bi_real

C:\Users\moham\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
18    -0.073455 -0.077940       1        1  13.811926
6      0.026971 -0.152917       2        1   8.275555
2      0.136171  0.013560       3        1   7.328012
16     0.135921  0.007865       4        1   7.275609
15     0.219412  0.066195       5        1   7.084424
13    -0.100211 -0.016814       6        1   5.462746
10     0.096503 -0.175039       7        1   5.451771
0      0.045394 -0.063355       8        1   5.198161
4     -0.088276  0.022229       9        1   5.063805
5     -0.047749  0.052090      10        1   4.777815
3      0.006819  0.007844      11        1   4.100080
17    -0.132207 -0.109797      12        1   3.862283
7     -0.127280  0.114228      13        1   3.861636
19     0.158828  0.047281      14        1   3.649733
14    -0.084439 -0.026303      15        1   3.183042
8     -0.102246 -0.071261      16        1   2.696435
1     -0.020518  0.066272      17        1   2.628154
9      0.010749  0.104914      18        1   2.581750
11     0.035172  0.037662      19        1   2.519817
12    -0.095559  0.153285      20        1   1.187245, topic_info=     Category         Freq        Term        Total  loglift  logprob
4046  Default  1390.000000        iran  1390.000000  30.0000  30.0000
5351  Default  1403.000000       party  1403.000000  29.0000  29.0000
1282  Default  1332.000000       china  1332.000000  28.0000  28.0000
3674  Default   912.000000      huawei   912.000000  27.0000  27.0000
958   Default   701.000000      brexit   701.000000  26.0000  26.0000
5717  Default  2027.000000   president  2027.000000  25.0000  25.0000
12    Default   609.000000    abortion   609.000000  24.0000  24.0000
2066  Default  1032.000000   democrats  1032.000000  23.0000  23.0000
7631  Default  2592.000000       think  2592.000000  22.0000  22.0000
8435  Default   856.000000        yeah   856.000000  21.0000  21.0000
7756  Default   966.000000       trade   966.000000  20.0000  20.0000
1489  Default  1332.000000   companies  1332.000000  19.0000  19.0000
2501  Default   910.000000    election   910.000000  18.0000  18.0000
1491  Default  1628.000000     company  1628.000000  17.0000  17.0000
1593  Default   909.000000    congress   909.000000  16.0000  16.0000
1284  Default   617.000000     chinese   617.000000  15.0000  15.0000
3515  Default   400.000000         hbo   400.000000  14.0000  14.0000
8208  Default  1377.000000         war  1377.000000  13.0000  13.0000
8169  Default   628.000000        vote   628.000000  12.0000  12.0000
6962  Default   362.000000       smith   362.000000  11.0000  11.0000
1953  Default  1021.000000        deal  1021.000000  10.0000  10.0000
4305  Default   882.000000         law   882.000000   9.0000   9.0000
3354  Default  1666.000000  government  1666.000000   8.0000   8.0000
4322  Default   587.000000      leader   587.000000   7.0000   7.0000
6625  Default   685.000000      school   685.000000   6.0000   6.0000
3664  Default  1132.000000       house  1132.000000   5.0000   5.0000
5307  Default   313.000000   pangolins   313.000000   4.0000   4.0000
7515  Default   394.000000     tariffs   394.000000   3.0000   3.0000
7159  Default  1486.000000       state  1486.000000   2.0000   2.0000
3620  Default   865.000000        home   865.000000   1.0000   1.0000
...       ...          ...         ...          ...      ...      ...
446   Topic20    20.256424     arizona    50.220720   3.5256  -6.0116
6664  Topic20   121.374011      season   308.819568   3.4997  -4.2212
1909  Topic20    22.409617    daenerys    58.282007   3.4777  -5.9106
1393  Topic20    28.528326       coach    80.779766   3.3927  -5.6692
1061  Topic20    85.969294  california   320.466599   3.1177  -4.5661
7430  Topic20    21.324213     suspect    63.133268   3.3481  -5.9603
3045  Topic20    19.476769       fires    61.862813   3.2778  -6.0509
5074  Topic20    37.910822    november 

In [92]:
bi_fake = pyLDAvis.sklearn.prepare(lda_tf_bi, dtm_tf_bi, tf_vectorizer_bi)
bi_fake

C:\Users\moham\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
14    -0.135249  0.130775       1        1  9.389228
3     -0.107612  0.187227       2        1  7.376079
17    -0.090957  0.038074       3        1  7.249380
7      0.132062  0.012122       4        1  7.198610
5     -0.090216 -0.060799       5        1  6.994337
1      0.003905  0.070874       6        1  6.952977
10     0.130452  0.013341       7        1  6.184997
6      0.091675 -0.008878       8        1  5.235826
8      0.196245  0.050783       9        1  5.118081
4     -0.039744 -0.131803      10        1  5.096601
0      0.163679 -0.057658      11        1  4.908223
16     0.057336  0.011084      12        1  4.535066
11     0.034447  0.086758      13        1  4.419459
18     0.101344  0.086927      14        1  4.239793
15    -0.121721 -0.073479      15        1  3.331421
13    -0.168956 -0.069693      16        1  3.303506
19     0.061960 -0.182589      17        1  2.441373
2     -0.151472  0.015151      18        1  2.182902
12     0.010968 -0.017285      19        1  2.168970
9     -0.078145 -0.100933      20        1  1.673169, topic_info=     Category         Freq         Term        Total  loglift  logprob
4353  Default  1392.000000         iran  1392.000000  30.0000  30.0000
3956  Default   990.000000       huawei   990.000000  29.0000  29.0000
6224  Default  2462.000000    president  2462.000000  28.0000  28.0000
1028  Default   743.000000       brexit   743.000000  27.0000  27.0000
12    Default   770.000000     abortion   770.000000  26.0000  26.0000
5828  Default  1360.000000        party  1360.000000  25.0000  25.0000
1386  Default  1334.000000        china  1334.000000  24.0000  24.0000
2216  Default  1047.000000    democrats  1047.000000  23.0000  23.0000
8277  Default  2589.000000        think  2589.000000  22.0000  22.0000
5782  Default   570.000000     pangolin   570.000000  21.0000  21.0000
8911  Default  1377.000000          war  1377.000000  20.0000  20.0000
1611  Default  1631.000000      company  1631.000000  19.0000  19.0000
4056  Default   644.000000  impeachment   644.000000  18.0000  18.0000
5878  Default   561.000000       pelosi   561.000000  17.0000  17.0000
3945  Default  1317.000000        house  1317.000000  16.0000  16.0000
2689  Default  1011.000000     election  1011.000000  15.0000  15.0000
1388  Default   824.000000      chinese   824.000000  14.0000  14.0000
9155  Default   855.000000         yeah   855.000000  13.0000  13.0000
5783  Default   432.000000    pangolins   432.000000  12.0000  12.0000
8411  Default  1035.000000        trade  1035.000000  11.0000  11.0000
3543  Default   578.000000        getty   578.000000  10.0000  10.0000
5892  Default  4277.000000       people  4277.000000   9.0000   9.0000
6815  Default  1247.000000       report  1247.000000   8.0000   8.0000
4656  Default   878.000000          law   878.000000   7.0000   7.0000
4782  Default  3464.000000         like  3464.000000   6.0000   6.0000
1608  Default  1346.000000    companies  1346.000000   5.0000   5.0000
7776  Default  1446.000000        state  1446.000000   4.0000   4.0000
1717  Default   954.000000     congress   954.000000   3.0000   3.0000
1435  Default   910.000000         city   910.000000   2.0000   2.0000
3480  Default   738.000000         game   738.000000   1.0000   1.0000
...       ...          ...          ...          ...      ...      ...
3231  Topic20    80.016009         film   181.186193   3.2732  -5.1147
1542  Topic20    17.545305     colonial    25.295997   3.7246  -6.6321
5313  Topic20    30.805833       movies    53.763425   3.5336  -6.0692
5608  Topic20    51.071318        omaha   111.139355   3.3129  -5.5637
8928  Topic20    75.200644         wars   220.526545   3.0146  -5.1768
7763  Topic20    87.937801         star   337.761764   2.7447  -5.0203
5931  Topic20    22.977321          pet    43.054311   3.4625  -6.3624
3766  Topic20    36.48

In [95]:
bi_all = pyLDAvis.sklearn.prepare(lda_tf_bi, dtm_tf_bi, tf_vectorizer_bi)
bi_all

C:\Users\moham\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.067063  0.013644       1        1  14.583870
9      0.049291  0.075364       2        1   8.875343
4     -0.162014  0.071112       3        1   8.099766
12    -0.019070 -0.141332       4        1   7.041265
10    -0.038290 -0.168442       5        1   6.645786
1     -0.045400  0.024986       6        1   6.350432
15     0.114533  0.052758       7        1   5.874884
2     -0.112232 -0.001618       8        1   5.383378
13    -0.184557  0.047027       9        1   5.002147
8      0.079664  0.099930      10        1   4.729740
3      0.093802  0.056855      11        1   4.380453
19    -0.109342  0.023106      12        1   3.429304
7      0.198171  0.042375      13        1   3.321443
17    -0.089015  0.067546      14        1   2.995755
16     0.104960 -0.024827      15        1   2.811047
5     -0.022289 -0.157497      16        1   2.799285
11     0.146579  0.071562      17        1   2.248857
14     0.099510 -0.195485      18        1   1.932286
18    -0.106153  0.023404      19        1   1.845379
6     -0.065212  0.019533      20        1   1.649581, topic_info=     Category         Freq            Term        Total  loglift  logprob
4005  Default  1363.000000            iran  1363.000000  30.0000  30.0000
8131  Default  1381.000000             war  1381.000000  29.0000  29.0000
5661  Default  2469.000000       president  2469.000000  28.0000  28.0000
1269  Default  1327.000000           china  1327.000000  27.0000  27.0000
3636  Default   875.000000          huawei   875.000000  26.0000  26.0000
5300  Default  1405.000000           party  1405.000000  25.0000  25.0000
1475  Default  1605.000000         company  1605.000000  24.0000  24.0000
12    Default   522.000000        abortion   522.000000  23.0000  23.0000
7562  Default  2581.000000           think  2581.000000  22.0000  22.0000
2045  Default  1012.000000       democrats  1012.000000  21.0000  21.0000
5469  Default   723.000000            plan   723.000000  20.0000  20.0000
2477  Default  1020.000000        election  1020.000000  19.0000  19.0000
4264  Default   893.000000             law   893.000000  18.0000  18.0000
948   Default   662.000000          brexit   662.000000  17.0000  17.0000
8293  Default   752.000000           women   752.000000  16.0000  16.0000
8357  Default   851.000000            yeah   851.000000  15.0000  15.0000
6374  Default  2363.000000           right  2363.000000  14.0000  14.0000
3717  Default   411.000000     immigration   411.000000  13.0000  13.0000
7684  Default   938.000000           trade   938.000000  12.0000  12.0000
5351  Default  4264.000000          people  4264.000000  11.0000  11.0000
1473  Default  1229.000000       companies  1229.000000  10.0000  10.0000
1932  Default  1107.000000            deal  1107.000000   9.0000   9.0000
7094  Default  1495.000000           state  1495.000000   8.0000   8.0000
1271  Default   625.000000         chinese   625.000000   7.0000   7.0000
4723  Default   579.000000        military   579.000000   6.0000   6.0000
1577  Default   879.000000        congress   879.000000   5.0000   5.0000
8358  Default  2049.000000            year  2049.000000   4.0000   4.0000
4368  Default  1191.000000            life  1191.000000   3.0000   3.0000
8360  Default  2162.000000           years  2162.000000   2.0000   2.0000
5518  Default   505.000000          police   505.000000   1.0000   1.0000
...       ...          ...             ...          ...      ...      ...
4009  Topic20    49.499217            iraq   246.742615   2.4983  -5.4449
308   Topic20    81.980259         america   577.262774   2.1528  -4.9404
311   Topic20    34.512798        americas   142.551256   2.6863  -5.8056
4005  Topic20   127.451512            iran  1363.638900   1.7345  -4.4992
5332  Topic20    38.064990           peace   174.651264   2.5812  -5.7076
5913  Topic20    55.202148     

In [104]:
tri_real = pyLDAvis.sklearn.prepare(lda_tfidf_tri, dtm_tfidf_tri, tfidf_vectorizer_tri)
tri_real

C:\Users\moham\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5      0.206652 -0.183082       1        1  33.933058
16     0.184695  0.192211       2        1  29.630847
3     -0.081231  0.123520       3        1  12.330938
8     -0.136272  0.014906       4        1   6.572169
6     -0.142052 -0.100111       5        1   6.520049
13     0.009573 -0.000720       6        1   1.223411
4     -0.009606 -0.016188       7        1   1.127725
0     -0.012922 -0.001894       8        1   0.895484
18    -0.004534 -0.005000       9        1   0.881051
1      0.007359 -0.003286      10        1   0.831348
11    -0.004097 -0.001942      11        1   0.788138
10     0.008054  0.001173      12        1   0.744949
9      0.002349 -0.004738      13        1   0.728160
2      0.003574 -0.006950      14        1   0.722348
7     -0.002197 -0.005860      15        1   0.656911
14    -0.008478 -0.000455      16        1   0.522245
17    -0.005275 -0.000588      17        1   0.498346
15    -0.005445 -0.000183      18        1   0.467806
12    -0.004828 -0.000820      19        1   0.465448
19    -0.005321  0.000009      20        1   0.459568, topic_info=     Category       Freq            Term      Total  loglift  logprob
6431  Default  23.000000           right  23.000000  30.0000  30.0000
4976  Default  16.000000            need  16.000000  29.0000  29.0000
1491  Default  31.000000         company  31.000000  28.0000  28.0000
5717  Default  37.000000       president  37.000000  27.0000  27.0000
300   Default  11.000000          amazon  11.000000  26.0000  26.0000
7630  Default  16.000000          things  16.000000  25.0000  25.0000
4133  Default  11.000000            john  11.000000  24.0000  24.0000
785   Default  16.000000             big  16.000000  23.0000  23.0000
4424  Default  36.000000            like  36.000000  22.0000  22.0000
8395  Default  26.000000           world  26.000000  21.0000  21.0000
4238  Default  19.000000            know  19.000000  20.0000  20.0000
6594  Default  20.000000             say  20.000000  19.0000  19.0000
6664  Default  10.000000          season  10.000000  18.0000  18.0000
8053  Default  11.000000           using  11.000000  17.0000  17.0000
7895  Default  12.000000         twitter  12.000000  16.0000  16.0000
5634  Default  15.000000            post  15.000000  15.0000  15.0000
126   Default  20.000000  administration  20.000000  14.0000  14.0000
2923  Default  14.000000             far  14.000000  13.0000  13.0000
769   Default  12.000000            best  12.000000  12.0000  12.0000
4046  Default  36.000000            iran  36.000000  11.0000  11.0000
8438  Default  29.000000           years  29.000000  10.0000  10.0000
644   Default   9.000000             ban   9.000000   9.0000   9.0000
8386  Default  20.000000            work  20.000000   8.0000   8.0000
4503  Default  13.000000            look  13.000000   7.0000   7.0000
5971  Default   8.000000       questions   8.000000   6.0000   6.0000
1593  Default  20.000000        congress  20.000000   5.0000   5.0000
4768  Default  14.000000        military  14.000000   4.0000   4.0000
8203  Default  21.000000            want  21.000000   3.0000   3.0000
7229  Default  14.000000           story  14.000000   2.0000   2.0000
5012  Default  34.000000             new  34.000000   1.0000   1.0000
...       ...        ...             ...        ...      ...      ...
7207  Topic20   0.011928      stockpiles   0.607256   1.4526  -9.0473
6619  Topic20   0.011928        scheming   0.610590   1.4471  -9.0473
1193  Topic20   0.011928        centered   1.357978   0.6478  -9.0473
1819  Topic20   0.011928         creator   0.954799   1.0000  -9.0473
7564  Topic20   0.011928      television   4.705434  -0.5949  -9.0473
1239  Topic20   0.011928       character   3.226660  -0.2177  -9.0473
626   Topic20   0.011928        backward   0.885025   1.0759  -9.0473
983   Topic20   0.011928    broadcastin

In [108]:
tri_fake = pyLDAvis.sklearn.prepare(lda_tfidf_tri, dtm_tfidf_tri, tfidf_vectorizer_tri)
tri_fake

C:\Users\moham\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
10     0.198422  0.171535       1        1  44.059176
4      0.108063 -0.243430       2        1  33.365419
14    -0.206815  0.029954       3        1   4.463958
9     -0.055115  0.022906       4        1   2.243914
13    -0.001314 -0.018299       5        1   2.023958
7     -0.001859  0.005532       6        1   1.725936
11    -0.011044  0.001424       7        1   1.596443
0     -0.008895 -0.002114       8        1   1.448613
17    -0.001796 -0.005380       9        1   1.391746
5     -0.014695  0.017302      10        1   1.356033
12     0.000768  0.001391      11        1   0.745280
6     -0.000446 -0.001698      12        1   0.731283
1     -0.012303  0.005063      13        1   0.721131
16     0.002650 -0.000225      14        1   0.658353
2      0.003736  0.003682      15        1   0.655711
15    -0.002456  0.001899      16        1   0.612298
8      0.000084  0.003045      17        1   0.612097
3      0.003462  0.002501      18        1   0.575840
19    -0.001318  0.003063      19        1   0.533269
18     0.000872  0.001847      20        1   0.479542, topic_info=     Category       Freq         Term      Total  loglift  logprob
4186  Default  17.000000  information  17.000000  30.0000  30.0000
6078  Default  17.000000       policy  17.000000  29.0000  29.0000
3956  Default  15.000000       huawei  15.000000  28.0000  28.0000
7339  Default  15.000000      service  15.000000  27.0000  27.0000
1028  Default  16.000000       brexit  16.000000  26.0000  26.0000
8228  Default  13.000000        terms  13.000000  25.0000  25.0000
6954  Default  12.000000       review  12.000000  24.0000  24.0000
12    Default  20.000000     abortion  20.000000  23.0000  23.0000
3731  Default  12.000000       happen  12.000000  22.0000  22.0000
1071  Default  10.000000      browser  10.000000  21.0000  21.0000
1848  Default  10.000000       cookie  10.000000  20.0000  20.0000
4420  Default  10.000000   javascript  10.000000  19.0000  19.0000
8018  Default  10.000000     supports  10.000000  18.0000  18.0000
1611  Default  23.000000      company  23.000000  17.0000  17.0000
1849  Default  10.000000      cookies  10.000000  16.0000  16.0000
910   Default  10.000000     blocking  10.000000  15.0000  15.0000
1388  Default  14.000000      chinese  14.000000  14.0000  14.0000
4379  Default  13.000000       israel  13.000000  13.0000  13.0000
4837  Default  10.000000      loading  10.000000  12.0000  12.0000
5409  Default  14.000000         need  14.000000  11.0000  11.0000
9111  Default  24.000000        world  24.000000  10.0000  10.0000
1115  Default  17.000000     business  17.000000   9.0000   9.0000
2097  Default  21.000000         deal  21.000000   8.0000   8.0000
1386  Default  25.000000        china  25.000000   7.0000   7.0000
6991  Default  21.000000        right  21.000000   6.0000   6.0000
4162  Default  11.000000     industry  11.000000   5.0000   5.0000
5450  Default  17.000000         news  17.000000   4.0000   4.0000
6224  Default  38.000000    president  38.000000   3.0000   3.0000
8276  Default  14.000000       things  14.000000   2.0000   2.0000
4580  Default  18.000000         know  18.000000   1.0000   1.0000
...       ...        ...          ...        ...      ...      ...
6336  Topic20   0.079690  projections   0.714082   3.1472  -7.2283
5398  Topic20   0.095997         neal   0.925598   3.0740  -7.0421
8851  Topic20   0.070648        vista   0.753981   2.9724  -7.3487
299   Topic20   0.073763         alma   0.853029   2.8922  -7.3056
9060  Topic20   0.041931       wiping   0.526321   2.8102  -7.8704
557   Topic20   0.039758    assurance   0.552156   2.7091  -7.9236
2841  Topic20   0.046146  enthusiasts   0.689128   2.6365  -7.7746
5889  Topic20   0.061436      pension   0.971923   2.5788  -7.4884
5278  Topic20   0.080913    morehouse   1.318894   2.5489  -7.2130
613   Topic20   0.0660

In [111]:
tri_all =  pyLDAvis.sklearn.prepare(lda_tfidf_tri, dtm_tfidf_tri, tfidf_vectorizer_tri)
tri_all

C:\Users\moham\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
15     0.257059 -0.053854       1        1  58.507440
9     -0.166877 -0.251882       2        1  16.127912
0     -0.099031  0.139429       3        1   5.152452
4     -0.104509  0.092640       4        1   4.014350
13    -0.060501  0.067503       5        1   3.594301
7      0.025932 -0.005456       6        1   1.587579
10    -0.001819  0.025359       7        1   1.575688
6      0.017603 -0.013073       8        1   1.034293
19     0.017351  0.004309       9        1   0.923162
8      0.014410  0.002442      10        1   0.862575
11     0.020944 -0.005943      11        1   0.786474
1      0.020430 -0.005167      12        1   0.783119
5      0.019535 -0.003627      13        1   0.780694
14     0.019785 -0.002041      14        1   0.758644
12    -0.000073  0.003347      15        1   0.707605
16     0.008741  0.003274      16        1   0.690739
3      0.000782  0.001368      17        1   0.619690
18     0.003420  0.000722      18        1   0.507595
2      0.004056 -0.000480      19        1   0.497499
17     0.002763  0.001130      20        1   0.488189, topic_info=     Category       Freq          Term      Total  loglift  logprob
3636  Default  15.000000        huawei  15.000000  30.0000  30.0000
8358  Default  30.000000          year  30.000000  29.0000  29.0000
1475  Default  26.000000       company  26.000000  28.0000  28.0000
8360  Default  29.000000         years  29.000000  27.0000  27.0000
948   Default  18.000000        brexit  18.000000  26.0000  26.0000
7607  Default  32.000000          time  32.000000  25.0000  25.0000
1269  Default  28.000000         china  28.000000  24.0000  24.0000
4930  Default  17.000000          need  17.000000  23.0000  23.0000
4005  Default  34.000000          iran  34.000000  22.0000  22.0000
4380  Default  37.000000          like  37.000000  21.0000  21.0000
5661  Default  42.000000     president  42.000000  20.0000  20.0000
5351  Default  44.000000        people  44.000000  19.0000  19.0000
5300  Default  25.000000         party  25.000000  18.0000  18.0000
3308  Default  18.000000          good  18.000000  17.0000  17.0000
12    Default  12.000000      abortion  12.000000  16.0000  16.0000
4197  Default  20.000000          know  20.000000  15.0000  15.0000
1312  Default  16.000000          city  16.000000  14.0000  14.0000
4264  Default  17.000000           law  17.000000  13.0000  13.0000
8317  Default  28.000000         world  28.000000  12.0000  12.0000
6374  Default  24.000000         right  24.000000  11.0000  11.0000
785   Default  16.000000       billion  16.000000  10.0000  10.0000
300   Default  11.000000        amazon  11.000000   9.0000   9.0000
6534  Default  20.000000           say  20.000000   8.0000   8.0000
1271  Default  13.000000       chinese  13.000000   7.0000   7.0000
777   Default  16.000000           big  16.000000   6.0000   6.0000
1473  Default  19.000000     companies  19.000000   5.0000   5.0000
4092  Default  11.000000          john  11.000000   4.0000   4.0000
4729  Default  20.000000       million  20.000000   3.0000   3.0000
4965  Default  34.000000           new  34.000000   2.0000   2.0000
7561  Default  16.000000        things  16.000000   1.0000   1.0000
...       ...        ...           ...        ...      ...      ...
2287  Topic20   0.117698           dna   2.010569   2.4842  -6.8176
2709  Topic20   0.102863     evolution   2.158078   2.2786  -6.9523
7930  Topic20   0.043215        unsafe   0.923700   2.2600  -7.8196
5172  Topic20   0.098543    originally   2.259679   2.1897  -6.9952
7999  Topic20   0.027042        varied   0.696638   2.0733  -8.2883
3446  Topic20   0.153077     happening   4.006872   2.0574  -6.5548
4084  Topic20   0.012552           jin   0.493413   1.6507  -9.0559
8387  Topic20   0.012552          zoos   0.493413   1.6507  -9.0559
1355  Topic20   0.012552    clinically   0.540037